# Decision tree Classification

## Preperation 

### Importing needed libraries

In [2]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
import preprocessing as pre

/Users/aaronsteiner/Documents/GitHub/Data_mining/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/aaronsteiner/Documents/GitHub/Data_mining/src/models/preprocessing.py:18: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aaronsteiner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aaronsteiner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aaronsteiner/nltk_data...
[nltk_data] 

### Loading Data

In [ ]:
tfidf, df_train, df_test = pre.setup(do_emojis=True)


Using custom data configuration default
Reusing dataset tweets_hate_speech_detection (/Users/aaronsteiner/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2)
100%|██████████| 1/1 [00:00<00:00, 24.09it/s]


In [84]:
df_train.head()

,label,preprocessed
30405,0,"[everyon, free, your, exam, mode, wtf, badtim,..."
27807,0,"[jacksonvil, rooster, simul, want, climb, vast..."
8660,0,"[user, run, 10km, user, user, loveisal, pour, ..."
19185,0,"[user, got, prototyp, new, usb, today, think, ..."
10355,0,"[amp, healthi, fathersday, runnerdad, eathealt..."


In [2]:
"""train_data = pd.read_csv("../../data/220510_train_data_preprocessed.csv", sep=';')
test_data = pd.read_csv("../../data/220510_test_data_preprocessed.csv", sep=";")
train_data.head()"""

,id,label,tweet,tweet_converted_emojis,n_mentions,hashtags,without_punctuation,tweet_lower,tweet_token,clean_token,clean_hashtags,stemmed_tokens,stemmed_hashtags,lemmatized_tokens,lemmatized_hashtags
0,1,0,@user when a father is dysfunctional and is s...,@user when a father is dysfunctional and is s...,1,['run'],user when a father is dysfunctional and is so...,user when a father is dysfunctional and is so...,"['user', 'when', 'a', 'father', 'is', 'dysfunc...","['user', 'father', 'dysfunctional', 'selfish',...",['run'],"['user', 'father', 'dysfunct', 'selfish', 'dra...",['run'],"['user', 'father', 'dysfunctional', 'selfish',...",['run']
1,2,0,@user @user thanks for #lyft credit i can't us...,@user @user thanks for #lyft credit i can't us...,2,"['lyft', 'disapointed', 'getthanked']",user user thanks for lyft credit i cant use ca...,user user thanks for lyft credit i cant use ca...,"['user', 'user', 'thanks', 'for', 'lyft', 'cre...","['user', 'user', 'thanks', 'lyft', 'credit', '...","['lyft', 'disapointed', 'getthanked']","['user', 'user', 'thank', 'lyft', 'credit', 'c...","['lyft', 'disapoint', 'getthank']","['user', 'user', 'thanks', 'lyft', 'credit', '...","['lyft', 'disapointed', 'getthanked']"
2,3,0,bihday your majesty,bihday your majesty,0,[],bihday your majesty,bihday your majesty,"['bihday', 'your', 'majesty']","['bihday', 'majesty']",[],"['bihday', 'majesti']",[],"['bihday', 'majesty']",[]
3,4,0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...,0,['model'],model i love u take with u all the time in u...,model i love u take with u all the time in u...,"['model', 'i', 'love', 'u', 'take', 'with', 'u...","['model', 'love', 'u', 'take', 'u', 'time', 'u...",['model'],"['model', 'love', 'u', 'take', 'u', 'time', 'u...",['model'],"['model', 'love', 'u', 'take', 'u', 'time', 'u...",['model']
4,5,0,factsguide: society now #motivation,factsguide: society now #motivation,0,['motivation'],factsguide society now motivation,factsguide society now motivation,"['factsguide', 'society', 'now', 'motivation']","['factsguide', 'society', 'motivation']",['motivation'],"['factsguid', 'societi', 'motiv']",['motiv'],"['factsguide', 'society', 'motivation']",['motivation']


## Feature Selection Lemmatized

In [3]:
"""features = ["n_mentions", "lemmatized_hashtags", "lemmatized_tokens"]
X = data[features]
y = data.label
X.head()"""

'features = ["n_mentions", "lemmatized_hashtags", "lemmatized_tokens"]\nX = data[features]\ny = data.label\nX.head()'

## Bag of words model

Create the bag of words from the data

In [4]:
"""import texthero as hero
from texthero import preprocessing
X["bow_tokens"] = (hero.tfidf(X["lemmatized_tokens"], max_features=15000))
X["bow_hashtags"] = (hero.tfidf(X["lemmatized_hashtags"], max_features=15000))
X"""

'import texthero as hero\nfrom texthero import preprocessing\nX["bow_tokens"] = (hero.tfidf(X["lemmatized_tokens"], max_features=15000))\nX["bow_hashtags"] = (hero.tfidf(X["lemmatized_hashtags"], max_features=15000))\nX'

Reduce the dimension of the vector

In [5]:
"""X["bow_tokens"] = (hero.tsne(X["bow_tokens"]))
X["bow_hashtags"] = (hero.tsne(X["bow_hashtags"]))
X"""

'X["bow_tokens"] = (hero.tsne(X["bow_tokens"]))\nX["bow_hashtags"] = (hero.tsne(X["bow_hashtags"]))\nX'

Split vector into columns

In [6]:
"""X[["bow_tokens_1", "bow_tokens_"]] = pd.DataFrame(X.bow_tokens.tolist(), index= X.index)
X[["bow_hashtags_1", "bow_hashtags_2"]] = pd.DataFrame(X.bow_hashtags.tolist(), index= X.index)
X"""

'X[["bow_tokens_1", "bow_tokens_"]] = pd.DataFrame(X.bow_tokens.tolist(), index= X.index)\nX[["bow_hashtags_1", "bow_hashtags_2"]] = pd.DataFrame(X.bow_hashtags.tolist(), index= X.index)\nX'

# Lemmatisation

In [7]:
"""features = ["lemmatized_tokens", "lemmatized_hashtags", "n_mentions"]

X = train_data[features]
y = train_data.label

X.head()"""

,lemmatized_tokens,lemmatized_hashtags,n_mentions
0,"['user', 'father', 'dysfunctional', 'selfish',...",['run'],1
1,"['user', 'user', 'thanks', 'lyft', 'credit', '...","['lyft', 'disapointed', 'getthanked']",2
2,"['bihday', 'majesty']",[],0
3,"['model', 'love', 'u', 'take', 'u', 'time', 'u...",['model'],0
4,"['factsguide', 'society', 'motivation']",['motivation'],0


### Using TF-IDF Vectorizer

In [8]:
X.shape

(31962, 3)

In [9]:
tf = TfidfVectorizer()

X_vec = tf.fit(X["lemmatized_tokens"])
X_lem = X_vec.transform(X["lemmatized_tokens"])

In [10]:
X_lem.shape

(31962, 40632)

## Split data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_lem, y, test_size=0.2, random_state = 17, stratify=y)

In [12]:
print(f"Shape X_train {X_train.shape}")
print(f"Shape X_test {X_test.shape}")

Shape X_train (25569, 40632)
Shape X_test (6393, 40632)


## Fitting model

In [25]:
result_precision_score = []
result_recall_score = []
result_accuracy_score = []
result_f1_score = []
max_depth = 100

for i in range(5, max_depth):
    classifier = DecisionTreeClassifier(random_state=55, max_depth=i)

    model = classifier.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    result_precision_score.append(metrics.precision_score(y_test, y_pred))
    result_recall_score.append(metrics.recall_score(y_test, y_pred))
    result_accuracy_score.append(metrics.accuracy_score(y_test, y_pred))
    result_f1_score.append(metrics.f1_score(y_test, y_pred))


#print(len([i for i in range(5, max_depth)]), len(result_precision_score),len(result_recall_score),len(result_accuracy_score),len(result_f1_score))


result_data = {
    "max_depth": [i for i in range(5, max_depth)],
    "precision_score": result_precision_score,
    "recall_score": result_recall_score,
    "accuracy_score": result_accuracy_score,
    "f1_score": result_f1_score
}
df = pd.DataFrame(result_data)
df

,max_depth,precision_score,recall_score,accuracy_score,f1_score
0,5,0.780142,0.245536,0.942281,0.373514
1,6,0.779221,0.267857,0.943376,0.398671
2,7,0.794118,0.301339,0.945565,0.436893
3,8,0.803681,0.292411,0.945409,0.428805
4,9,0.807018,0.308036,0.946348,0.445880
...,...,...,...,...,...
90,95,0.718182,0.529018,0.952448,0.609254
91,96,0.716049,0.517857,0.951822,0.601036
92,97,0.722045,0.504464,0.951666,0.593955
93,98,0.711246,0.522321,0.951666,0.602317


In [26]:
import plotly.express as px


fig = px.line(df, x="max_depth", y= ["f1_score", "accuracy_score"], title="Lemmatisation results")
fig.show()

[[1, 0.9278339764313276], [2, 0.9278339764313276], [3, 0.9296068411721764], [4, 0.9297111273334029], [5, 0.9311711335905726], [6, 0.9316925643967046], [7, 0.9320054228803838], [8, 0.9322139952028365], [9, 0.931275419751799], [10, 0.9327354260089686], [11, 0.932318281364063], [12, 0.9316925643967046], [13, 0.9298154134946293], [14, 0.9300239858170821], [15, 0.9292939826884973], [16, 0.9285639795599124], [17, 0.9292939826884973], [18, 0.9271039733027427], [19, 0.9248096777557618], [20, 0.9244968192720826], [21, 0.9206382313067056], [22, 0.9203253728230264], [23, 0.9161539263739702], [24, 0.9144853477943476], [25, 0.9147982062780269], [26, 0.911982479924914], [27, 0.9087496089268954], [28, 0.9072896026697257], [29, 0.9055167379288769], [30, 0.9042653039941599], [31, 0.9036395870268016], [32, 0.9040567316717072], [33, 0.9028052977369903], [34, 0.9028052977369903], [35, 0.903222442381896], [36, 0.9017624361247263], [37, 0.900928146834915], [38, 0.900928146834915], [39, 0.9011367191573678], [40, 0.9008238606736886], [41, 0.8987381374491605], [42, 0.8992595682552925], [43, 0.8989467097716133], [44, 0.8973824173532172], [45, 0.8958181249348212], [46, 0.8966524142246324], [47, 0.8961309834185004], [48, 0.8964438419021796], [49, 0.8957138387735948]]

Dataset split: 0.9278339764313276

## Try Up-sample Minority Class

Seperate Minortity Class

In [60]:
data_minority = train_data[train_data.label == 1]
data_majority = train_data[train_data.label == 0]
print("length majority", len(data_majority))
print("length minority", len(data_minority))

length majority 29720
length minority 2242


Upsample minority class

In [61]:
data_minority = resample(data_minority, replace = True, n_samples=10000, random_state=55)

Combine the classes

In [62]:
data_upsampled = pd.concat([data_majority, data_minority])
data_upsampled.label.value_counts()

0    29720
1    10000
Name: label, dtype: int64

In [63]:
data_upsampled.head()

,id,label,tweet,tweet_converted_emojis,n_mentions,hashtags,without_punctuation,tweet_lower,tweet_token,clean_token,clean_hashtags,stemmed_tokens,stemmed_hashtags,lemmatized_tokens,lemmatized_hashtags
0,1,0,@user when a father is dysfunctional and is s...,@user when a father is dysfunctional and is s...,1,['run'],user when a father is dysfunctional and is so...,user when a father is dysfunctional and is so...,"['user', 'when', 'a', 'father', 'is', 'dysfunc...","['user', 'father', 'dysfunctional', 'selfish',...",['run'],"['user', 'father', 'dysfunct', 'selfish', 'dra...",['run'],"['user', 'father', 'dysfunctional', 'selfish',...",['run']
1,2,0,@user @user thanks for #lyft credit i can't us...,@user @user thanks for #lyft credit i can't us...,2,"['lyft', 'disapointed', 'getthanked']",user user thanks for lyft credit i cant use ca...,user user thanks for lyft credit i cant use ca...,"['user', 'user', 'thanks', 'for', 'lyft', 'cre...","['user', 'user', 'thanks', 'lyft', 'credit', '...","['lyft', 'disapointed', 'getthanked']","['user', 'user', 'thank', 'lyft', 'credit', 'c...","['lyft', 'disapoint', 'getthank']","['user', 'user', 'thanks', 'lyft', 'credit', '...","['lyft', 'disapointed', 'getthanked']"
2,3,0,bihday your majesty,bihday your majesty,0,[],bihday your majesty,bihday your majesty,"['bihday', 'your', 'majesty']","['bihday', 'majesty']",[],"['bihday', 'majesti']",[],"['bihday', 'majesty']",[]
3,4,0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...,0,['model'],model i love u take with u all the time in u...,model i love u take with u all the time in u...,"['model', 'i', 'love', 'u', 'take', 'with', 'u...","['model', 'love', 'u', 'take', 'u', 'time', 'u...",['model'],"['model', 'love', 'u', 'take', 'u', 'time', 'u...",['model'],"['model', 'love', 'u', 'take', 'u', 'time', 'u...",['model']
4,5,0,factsguide: society now #motivation,factsguide: society now #motivation,0,['motivation'],factsguide society now motivation,factsguide society now motivation,"['factsguide', 'society', 'now', 'motivation']","['factsguide', 'society', 'motivation']",['motivation'],"['factsguid', 'societi', 'motiv']",['motiv'],"['factsguide', 'society', 'motivation']",['motivation']


In [64]:
features = ["lemmatized_tokens", "lemmatized_hashtags", "n_mentions"]

X_upsampled = data_upsampled[features]
y_upsampled = data_upsampled.label

X.head()

,lemmatized_tokens,lemmatized_hashtags,n_mentions
0,"['user', 'father', 'dysfunctional', 'selfish',...",['run'],1
1,"['user', 'user', 'thanks', 'lyft', 'credit', '...","['lyft', 'disapointed', 'getthanked']",2
2,"['bihday', 'majesty']",[],0
3,"['model', 'love', 'u', 'take', 'u', 'time', 'u...",['model'],0
4,"['factsguide', 'society', 'motivation']",['motivation'],0


### Using TF-IDF Vectorizer

In [65]:
X_upsampled.shape

(39720, 3)

In [73]:
tf = TfidfVectorizer()

X_vec = tf.fit(X_upsampled["lemmatized_tokens"])
X_lem_up = X_vec.transform(X_upsampled["lemmatized_tokens"])

In [67]:
X_lem_up.shape

(39720, 40603)

## Split data

In [68]:
X_train_up, X_test_up, y_train_up, y_test_up = train_test_split(X_lem_up, y_upsampled, test_size=0.2, random_state = 17, stratify=y_upsampled)

In [69]:
print(f"Shape X_train {X_train_up.shape}")
print(f"Shape X_test {X_test_up.shape}")

Shape X_train (31776, 40603)
Shape X_test (7944, 40603)


## Fitting model

In [70]:
result_precision_score = []
result_recall_score = []
result_accuracy_score = []
result_f1_score = []
max_depth = 100

for i in range(5, max_depth):
    classifier = DecisionTreeClassifier(random_state=55, max_depth=i)

    model = classifier.fit(X_train_up, y_train_up)

    y_pred = model.predict(X_test_up)
    
    result_precision_score.append(metrics.precision_score(y_test_up, y_pred))
    result_recall_score.append(metrics.recall_score(y_test_up, y_pred))
    result_accuracy_score.append(metrics.accuracy_score(y_test_up, y_pred))
    result_f1_score.append(metrics.f1_score(y_test_up, y_pred))


#print(len([i for i in range(5, max_depth)]), len(result_precision_score),len(result_recall_score),len(result_accuracy_score),len(result_f1_score))


result_data = {
    "max_depth": [i for i in range(5, max_depth)],
    "precision_score": result_precision_score,
    "recall_score": result_recall_score,
    "accuracy_score": result_accuracy_score,
    "f1_score": result_f1_score
}
df_upsampled = pd.DataFrame(result_data)
df_upsampled

,max_depth,precision_score,recall_score,accuracy_score,f1_score
0,5,0.927273,0.2550,0.807402,0.400000
1,6,0.792497,0.3380,0.811052,0.473887
2,7,0.936982,0.2825,0.814577,0.434114
3,8,0.943114,0.3150,0.822759,0.472264
4,9,0.911528,0.3400,0.825529,0.495266
...,...,...,...,...,...
90,95,0.910137,0.8305,0.936682,0.868497
91,96,0.913925,0.8335,0.938318,0.871862
92,97,0.916072,0.8350,0.939199,0.873659
93,98,0.913162,0.8360,0.938696,0.872879


In [71]:
import plotly.express as px


fig = px.line(df_upsampled, x="max_depth", y= ["f1_score", "accuracy_score"], title="Lemmatisation results upsampeled")
fig.show()

# Grid search

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

## Setup parameters

In [77]:
X.head()

<bound method NDFrame.head of                                        lemmatized_tokens  \
0      ['user', 'father', 'dysfunctional', 'selfish',...   
1      ['user', 'user', 'thanks', 'lyft', 'credit', '...   
2                                  ['bihday', 'majesty']   
3      ['model', 'love', 'u', 'take', 'u', 'time', 'u...   
4                ['factsguide', 'society', 'motivation']   
...                                                  ...   
31957  ['ate', 'user', 'isz', 'youuusmiling', 'face',...   
31958  ['see', 'nina', 'turner', 'airwave', 'trying',...   
31959  ['listening', 'sad', 'song', 'monday', 'mornin...   
31960  ['user', 'sikh', 'temple', 'vandalised', 'calg...   
31961                        ['thank', 'user', 'follow']   

                         lemmatized_hashtags  n_mentions  
0                                    ['run']           1  
1      ['lyft', 'disapointed', 'getthanked']           2  
2                                         []           0  
3            

In [94]:
max_depth = 130
criterion = ['gini', 'entropy']
max_depth = [i for i in range(5, max_depth, 5)]

#min_samples_split
#min_samples_leaf
#min_weight_fraction_leaf
#max_featuresint, float or {“auto”, “sqrt”, “log2”}, default=None


parameters = dict(dec_tree__criterion=criterion,
                  dec_tree__max_depth=max_depth)


## Pipline

In [4]:
dec_tree = DecisionTreeClassifier(random_state=55)

pipe = Pipeline(steps=[('dec_tree', dec_tree)])

## Search

In [96]:
clf_GS = GridSearchCV(pipe, parameters)
clf_GS.fit(X_train, y_train)

print('Best Criterion:', clf_GS.best_estimator_.get_params()['dec_tree__criterion'])
print('Best max_depth:', clf_GS.best_estimator_.get_params()['dec_tree__max_depth'])
print(clf_GS.best_estimator_.get_params()['dec_tree'])


Best Criterion: gini
Best max_depth: 55
DecisionTreeClassifier(max_depth=55, random_state=55)


# With Preprocessing.py

### Loading Data

In [5]:
tfidf, df_train, df_test = pre.setup(do_emojis=True)


Using custom data configuration default
Reusing dataset tweets_hate_speech_detection (/Users/aaronsteiner/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2)
100%|██████████| 1/1 [00:00<00:00, 58.75it/s]


In [6]:
df_train.head()

,label,preprocessed
30405,0,"[everyon, free, your, exam, mode, wtf, badtim,..."
27807,0,"[jacksonvil, rooster, simul, want, climb, vast..."
8660,0,"[user, run, 10km, user, user, loveisal, pour, ..."
19185,0,"[user, got, prototyp, new, usb, today, think, ..."
10355,0,"[amp, healthi, fathersday, runnerdad, eathealt..."


In [7]:
def train_model(df_train: pd.DataFrame, tfidf: TfidfVectorizer):
    tree = DecisionTreeClassifier(random_state=55)

    pipe = Pipeline(steps=[('dec_tree', tree)])

    Xt_train = tfidf.transform(df_train['preprocessed'])
    y_train = df_train['label']
    max_depth = 300
    min_depth = 100
    criterion = ['gini', 'entropy']
    max_depth = [i for i in range(min_depth, max_depth, 5)]
    min_samples_split = [i for i in range(1, 10)]
    min_samples_leaf = [i for i in range(1, 10)]
    #min_weight_fraction_leaf = [i for i in range(0, 1, 0.1)]
    # max_featuresint, float or {“auto”, “sqrt”, “log2”}, default=None

    parameters = dict(dec_tree__criterion=criterion,
                      dec_tree__max_depth=max_depth, dec_tree__min_samples_split=min_samples_split,
                      dec_tree__min_samples_leaf=min_samples_leaf)

    dec_tree = GridSearchCV(pipe, param_grid=parameters, verbose=10)
    dec_tree.fit(Xt_train, y_train)

    return dec_tree.best_estimator_


In [8]:
def test_model(model, df_test: pd.DataFrame, tfidf: TfidfVectorizer):
    
    Xt_test = tfidf.transform(df_test['preprocessed'])
    y_test = df_test['label']
    y_pred = model.predict(Xt_test)

    predictions = []

    predictions.append(model.get_params())
    predictions.append(metrics.precision_score(y_test, y_pred))
    predictions.append(metrics.recall_score(y_test, y_pred))
    predictions.append(metrics.accuracy_score(y_test, y_pred))
    predictions.append(metrics.f1_score(y_test, y_pred))

    return predictions

In [9]:
model = train_model(df_train, tfidf)
results = test_model(model, df_test, tfidf)

print(results)

Fitting 5 folds for each of 6480 candidates, totalling 32400 fits
[CV 1/5; 1/6480] START dec_tree__criterion=gini, dec_tree__max_depth=100, dec_tree__min_samples_leaf=1, dec_tree__min_samples_split=1
[CV 1/5; 1/6480] END dec_tree__criterion=gini, dec_tree__max_depth=100, dec_tree__min_samples_leaf=1, dec_tree__min_samples_split=1;, score=nan total time=   0.0s
[CV 2/5; 1/6480] START dec_tree__criterion=gini, dec_tree__max_depth=100, dec_tree__min_samples_leaf=1, dec_tree__min_samples_split=1
[CV 2/5; 1/6480] END dec_tree__criterion=gini, dec_tree__max_depth=100, dec_tree__min_samples_leaf=1, dec_tree__min_samples_split=1;, score=nan total time=   0.0s
[CV 3/5; 1/6480] START dec_tree__criterion=gini, dec_tree__max_depth=100, dec_tree__min_samples_leaf=1, dec_tree__min_samples_split=1
[CV 3/5; 1/6480] END dec_tree__criterion=gini, dec_tree__max_depth=100, dec_tree__min_samples_leaf=1, dec_tree__min_samples_split=1;, score=nan total time=   0.0s
[CV 4/5; 1/6480] START dec_tree__criterion=

KeyboardInterrupt: 

{'memory': None, 'steps': [('dec_tree', DecisionTreeClassifier(criterion='entropy', max_depth=195, random_state=55))], 'verbose': False, 'dec_tree': DecisionTreeClassifier(criterion='entropy', max_depth=195, random_state=55), 'dec_tree__ccp_alpha': 0.0, 'dec_tree__class_weight': None, 'dec_tree__criterion': 'entropy', 'dec_tree__max_depth': 195, 'dec_tree__max_features': None, 'dec_tree__max_leaf_nodes': None, 'dec_tree__min_impurity_decrease': 0.0, 'dec_tree__min_samples_leaf': 1, 'dec_tree__min_samples_split': 2, 'dec_tree__min_weight_fraction_leaf': 0.0, 'dec_tree__random_state': 55, 'dec_tree__splitter': 'best'}
{'cv': None, 'error_score': nan, 'estimator__memory': None, 'estimator__steps': [('dec_tree', DecisionTreeClassifier(random_state=55))], 'estimator__verbose': False, 'estimator__dec_tree': DecisionTreeClassifier(random_state=55), 'estimator__dec_tree__ccp_alpha': 0.0, 'estimator__dec_tree__class_weight': None, 'estimator__dec_tree__criterion': 'gini', 'estimator__dec_tree__max_depth': None, 'estimator__dec_tree__max_features': None, 'estimator__dec_tree__max_leaf_nodes': None, 'estimator__dec_tree__min_impurity_decrease': 0.0, 'estimator__dec_tree__min_samples_leaf': 1, 'estimator__dec_tree__min_samples_split': 2, 'estimator__dec_tree__min_weight_fraction_leaf': 0.0, 'estimator__dec_tree__random_state': 55, 'estimator__dec_tree__splitter': 'best', 'estimator': Pipeline(steps=[('dec_tree', DecisionTreeClassifier(random_state=55))]), 'n_jobs': None, 'param_grid': {'dec_tree__criterion': ['gini', 'entropy'], 'dec_tree__max_depth': [100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195]}, 'pre_dispatch': '2*n_jobs', 'refit': True, 'return_train_score': False, 'scoring': None, 'verbose': 0}
[0.5521064301552107, 0.5558035714285714, 0.9372751446895041, 0.5539488320355951]

{'memory': None, 'steps': [('dec_tree', DecisionTreeClassifier(criterion='entropy', max_depth=220, random_state=55))], 'verbose': False, 'dec_tree': DecisionTreeClassifier(criterion='entropy', max_depth=220, random_state=55), 'dec_tree__ccp_alpha': 0.0, 'dec_tree__class_weight': None, 'dec_tree__criterion': 'entropy', 'dec_tree__max_depth': 220, 'dec_tree__max_features': None, 'dec_tree__max_leaf_nodes': None, 'dec_tree__min_impurity_decrease': 0.0, 'dec_tree__min_samples_leaf': 1, 'dec_tree__min_samples_split': 2, 'dec_tree__min_weight_fraction_leaf': 0.0, 'dec_tree__random_state': 55, 'dec_tree__splitter': 'best'}
[0.5626423690205011, 0.5513392857142857, 0.9385265133740028, 0.5569334836527621]